In [6]:
import requests
from bs4 import BeautifulSoup
import json
from queue import Queue
from urllib.parse import urljoin
# URL of the webpage
url = "https://www.screener.in/explore/"

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save raw HTML in a variable
    html = response.text
base_url = "https://www.screener.in"  # Replace with the actual base URL

soup = BeautifulSoup(html, 'html.parser')
sector_links = soup.find_all('a', class_='bordered radius-6 padding-4-12 font-size-14 ink-700')

print(f"Found {len(sector_links)} sector links")

sectors_queue = Queue()

for link in sector_links:
    href = link.get('href')
    absolute_url = urljoin(base_url, href)
    print(f"Processing URL: {absolute_url}")
    
    sectors_queue.put(absolute_url)

print(f"Queue size: {sectors_queue.qsize()}")

queue_list = []
while not sectors_queue.empty():
    queue_list.append(sectors_queue.get())

with open('sectors_queue.json', 'w') as f:
    json.dump(queue_list, f)
    print("Queue saved to sectors_queue.json")

Found 86 sector links
Processing URL: https://www.screener.in/company/compare/00000085/
Processing URL: https://www.screener.in/company/compare/00000001/
Processing URL: https://www.screener.in/company/compare/00000002/
Processing URL: https://www.screener.in/company/compare/00000003/
Processing URL: https://www.screener.in/company/compare/00000004/
Processing URL: https://www.screener.in/company/compare/00000005/
Processing URL: https://www.screener.in/company/compare/00000006/
Processing URL: https://www.screener.in/company/compare/00000066/
Processing URL: https://www.screener.in/company/compare/00000007/
Processing URL: https://www.screener.in/company/compare/00000008/
Processing URL: https://www.screener.in/company/compare/00000009/
Processing URL: https://www.screener.in/company/compare/00000010/
Processing URL: https://www.screener.in/company/compare/00000011/
Processing URL: https://www.screener.in/company/compare/00000012/
Processing URL: https://www.screener.in/company/compar

In [10]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import time
import random
import logging
from queue import Queue

# # Load HTML file for testing purposes. 
# with open('Aerospace & Defence Companies - Screener.htm', 'r', encoding='utf-8') as f:
#     htmltest = f.read()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_company_links(html):
    # Load HTML file
    soup = BeautifulSoup(html, 'html.parser')

    # Find all company links
    company_links = soup.find_all('a', target="_blank")

    # Create a queue to hold company URLs
    company_queue = Queue()

    # Extract company URLs and add to queue
    for link in company_links:
        company_url = link.get('href')
        company_queue.put(company_url)
        logging.info(f"Added company URL: {company_url}")

    # Print queue size
    logging.info(f"Queue size: {company_queue.qsize()}")

    # Convert queue to list
    company_list = []
    while not company_queue.empty():
        company_list.append(company_queue.get())

    return company_list

In [16]:

def load_visited_sectors():
    try:
        logging.info("Reading visited sectors from visited_sectors.json")
        with open('visited_sectors.json', 'r') as f:
            return set(json.load(f))
    except FileNotFoundError:
        logging.warning("visited_sectors.json not found. Creating new file")
        return set()


def save_visited_sectors(visited_sectors):
    logging.info("Writing visited sectors to visited_sectors.json")
    with open('visited_sectors.json', 'w') as f:
        json.dump(list(visited_sectors), f, indent=4)
    logging.info("Visited sectors written successfully")


def is_sector_visited(sector_name, visited_sectors):
    return sector_name in visited_sectors


def mark_sector_as_visited(sector_name, visited_sectors):
    visited_sectors.append(sector_name)
    save_visited_sectors(visited_sectors)
    
    
def extract_sector_name(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    title_tag = soup.find('title')
    if title_tag:
        title = title_tag.text
        # Remove "- Screener" from the title
        sector_name = title.replace("- Screener", "").strip()
        return sector_name
    else:
        return None
    

In [ ]:
# import time
# import logging


# # Load sector URLs from JSON file
# with open('sectors_queue.json', 'r') as f:
#     sector_urls = json.load(f)


# start_time = time.time()
# timeout = 5  # seconds


# for sector_url in sector_urls:
#     # Check if timeout has been exceeded
#     elapsed_time = time.time() - start_time
#     if elapsed_time > timeout:
#         logging.error("Timeout: Execution stopped after {} seconds".format(timeout))
#         break


#     # Fetch HTML content
#     response = requests.get(sector_url)
#     html_content = response.content
    
#     sector_name = extract_sector_name(html_content)
    
#     # Check if sector has been visited
#     try:
#         with open('visited_sectors.json', 'r') as f:
#             visited_sectors = json.load(f)
#         if sector_name in visited_sectors:
#             logging.info(f"Skipping already visited sector: {sector_name}")
#             continue
#     except FileNotFoundError:
#         pass


#     logging.info(f"Processing sector: {sector_name}")
#     time.sleep(2)  # Simulate processing time

2024-11-08 23:53:28,971 - INFO - Skipping already visited sector: Aerospace & Defence Companies
2024-11-08 23:53:29,217 - INFO - Skipping already visited sector: Agro Chemicals Companies
2024-11-08 23:53:29,434 - INFO - Skipping already visited sector: Air Transport Service Companies
2024-11-08 23:53:29,677 - INFO - Skipping already visited sector: Alcoholic Beverages Companies
2024-11-08 23:53:29,929 - INFO - Skipping already visited sector: Auto Ancillaries Companies
2024-11-08 23:53:30,169 - INFO - Skipping already visited sector: Automobile Companies
2024-11-08 23:53:30,423 - INFO - Skipping already visited sector: Banks Companies
2024-11-08 23:53:30,643 - INFO - Skipping already visited sector: Bearings Companies
2024-11-08 23:53:30,890 - INFO - Skipping already visited sector: Cables Companies
2024-11-08 23:53:31,146 - INFO - Skipping already visited sector: Capital Goods - Electrical Equipment Companies
2024-11-08 23:53:31,398 - INFO - Processing sector: Capital Goods-Non Electr

In [20]:
def get_queue_from_sector(sector_urls,timeout):
    base_url = "https://www.screener.in"  # Replace with the actual base URL

    # User-agent rotation list
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:46.0) Gecko/20100101 Firefox/46.0'
    ]

    last_request_time = time.time()
    max_retries = 3
    time_interval = 15  # seconds
    start_time = time.time()
    company_data = {}

    # Iterate over sector URLs
    for sector_url in sector_urls:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout:
            logging.error("Timeout: Execution stopped after {} seconds".format(timeout))
            break
        # Fetch HTML content
        response = requests.get(sector_url)
        html_content = response.content
        
        sector_name = extract_sector_name(html_content)
        
        # Check if sector has been visited
        try:
            with open('visited_sectors.json', 'r') as f:
                visited_sectors = json.load(f)
            if sector_name in visited_sectors:
                logging.info(f"Skipping already visited sector: {sector_name}")
                continue
        except FileNotFoundError:
            pass


        company_data[sector_name] = []

        # Add limit=50 parameter to URL
        sector_url_with_limit = f"{sector_url}?limit=50"
        
        logging.info(f"Visiting URL: {sector_url_with_limit}")
        
        retries = 0
        while retries < max_retries:
            try:
                # Calculate delay to ensure time_interval gap between requests
                current_time = time.time()
                time_elapsed = current_time - last_request_time
                delay = max(0, time_interval - time_elapsed)
                logging.info(f"Waiting {delay} seconds before requesting...")
                time.sleep(delay)
                
                # Update last request time
                last_request_time = time.time()
                
                # Randomly select user-agent
                headers = {'User-Agent': random.choice(user_agents)}
                
                # Send HTTP request to sector URL
                response = requests.get(sector_url_with_limit, headers=headers, timeout=10)
                
                if response.status_code == 200:
                    logging.info(f"Successful request to {sector_url_with_limit}")
                    
                    # Extract sector name from HTML content
                    sector_name = extract_sector_name(response.content)
                    if sector_name:
                        # Initialize company_data dictionary with correct sector_name
                        if sector_name not in company_data:
                            company_data[sector_name] = []
                        
                        # Extract company links
                        companies = extract_company_links(response.content)
                        company_data[sector_name].extend(companies)
                        
                        # Save company URLs to JSON file
                        logging.info(f"Saving company data to company_queue.json")
                        with open('company_queue.json', 'w') as f:
                            json.dump(company_data, f, indent=4)
                        
                        # Mark sector as visited
                        mark_sector_as_visited(sector_name, visited_sectors)
                        
                        break  # Exit loop on success
                    else:
                        logging.error("Failed to extract sector name")
                elif response.status_code == 429:
                    logging.warning(f"Rate limit exceeded for {sector_url}. Waiting 60 seconds.")
                    time.sleep(60)  # Wait 60 seconds before retrying
                    retries += 1
                else:
                    logging.error(f"Error {response.status_code} for {sector_url}")
                    retries += 1
            except requests.exceptions.RequestException as e:
                logging.error(f"Request error: {e}")
                retries += 1
                time.sleep(1)  # Wait 1 second before retrying

In [ ]:
# Load sector URLs from JSON file
with open('sectors_queue.json', 'r') as f:
    sector_urls = json.load(f)


currCodeTimeout = 10
get_queue_from_sector(sector_urls, currCodeTimeout)

2024-11-08 23:23:56,722 - INFO - Skipping already visited sector: Aerospace & Defence Companies
2024-11-08 23:23:56,967 - INFO - Skipping already visited sector: Agro Chemicals Companies
2024-11-08 23:23:57,188 - INFO - Skipping already visited sector: Air Transport Service Companies
2024-11-08 23:23:57,425 - INFO - Skipping already visited sector: Alcoholic Beverages Companies
2024-11-08 23:23:57,660 - INFO - Skipping already visited sector: Auto Ancillaries Companies
2024-11-08 23:23:57,919 - INFO - Skipping already visited sector: Automobile Companies
2024-11-08 23:23:58,157 - INFO - Skipping already visited sector: Banks Companies
2024-11-08 23:23:58,374 - INFO - Skipping already visited sector: Bearings Companies
2024-11-08 23:23:58,601 - INFO - Skipping already visited sector: Cables Companies
2024-11-08 23:23:58,854 - INFO - Skipping already visited sector: Capital Goods - Electrical Equipment Companies
2024-11-08 23:23:59,091 - INFO - Skipping already visited sector: Capital Go

Timeout: Execution stopped after 5 minutes


2024-11-08 23:24:56,688 - INFO - Successful request to https://www.screener.in/company/compare/00000020/?limit=50
2024-11-08 23:24:56,789 - INFO - Added company URL: /company/TITAN/consolidated/
2024-11-08 23:24:56,791 - INFO - Added company URL: /company/KALYANKJIL/consolidated/
2024-11-08 23:24:56,793 - INFO - Added company URL: /company/PNGJL/
2024-11-08 23:24:56,795 - INFO - Added company URL: /company/SENCO/
2024-11-08 23:24:56,797 - INFO - Added company URL: /company/RAJESHEXPO/consolidated/
2024-11-08 23:24:56,797 - INFO - Added company URL: /company/PCJEWELLER/consolidated/
2024-11-08 23:24:56,798 - INFO - Added company URL: /company/THANGAMAYL/
2024-11-08 23:24:56,799 - INFO - Added company URL: /company/SKYGOLD/
2024-11-08 23:24:56,802 - INFO - Added company URL: /company/VAIBHAVGBL/consolidated/
2024-11-08 23:24:56,804 - INFO - Added company URL: /company/GOLDIAM/consolidated/
2024-11-08 23:24:56,806 - INFO - Added company URL: /company/MOTISONS/
2024-11-08 23:24:56,807 - IN